# Master TIDE - Conférences Python 2021

Francis Wolinski

&copy; 2021 Yotta Conseil

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline

# display options
pd.set_option("display.max_rows", 20)

# 10. Séries temporelles


Introduction aux séries temporelles avec Python et pandas.

Le module standard `datetime` permet de manipuler des données temporelles avec Python.

Il existe d'autres modules avec le même objectif: `calendar`, `dateutil`, `pytz`, ...

Le module `datetime` fournit différents types de données temporelles :

- **date**: date (année, mois, jour) dans le caledrier Grégorien
- **time**: temps non rattaché à une date (heures, minutes, secondes, microsecondes)
- **datetime**: horodatage (date + temps)
- **timedelta**: durée, difference entre 2 dates ou 2 temps (jours, heures, minutes, secondes, microsecondes)
- **tzinfo**: gestion des fuseaux horaires

In [ ]:
# import de la librairie datetime
import datetime

## 10.1 Objets temporels Python

### Date

In [ ]:
# date
today = datetime.date.today()
today

### Datetime

In [ ]:
# datetime
now = datetime.datetime.now()
now

In [ ]:
# datetime en temps universel
now_utc = datetime.datetime.utcnow()
now_utc

### Fuseau horaire

Le module standard `pytz` gère les fuseaux horaires.

Un objet de type `datetime` peut être converti avec la méthode `astimezone()` dans un objet `datetime` d'un autre fuseau horaire.

Il est recommandé de travailler en temps universel (UTC) pour pallier la difficulté de gérer des objets de temps de différents fuseaux horaires.

In [ ]:
# import pytz
from pytz import timezone

In [ ]:
# instance de fuseau horaire
tz = timezone('Asia/Shanghai')
tz

In [ ]:
# now
now

In [ ]:
# convertit une datetime avec un objet de type fuseau horaire
now.astimezone(tz)

## 10.2 Ecriture et lecture

La méthode `strftime()` et la fonction `strptime()`  permettent d'écrire et de lire des dates dans différents formats avec une codificaton empruntée au langage C.

Le nombre de directives disponibles révèle la complexité du sujet.

Directives | Commentaires
- | -
%a | Day of the week abbreviated
%A | Day of the week
%w | Day of the week 0 = Sunday ... 6 = Saturday
%d | Day of month on 2 digits 01, 02, ..., 31
%j | Day of year on 3 digits 001, 002, ..., 366
%b | Month abbreviated
%B | Month name
%m | Month on 2 digits 01, 02, ..., 12
%U | Number of week in year (Sunday = first day)
%W | Number of week in year (Monday = first day)
%y | Year without the century on 2 digits 00, 01, ..., 99	 
%Y | Year with the century on 4 digits 0001, 0002, ..., 2018, 2019, ..., 9998, 9999
%H | Hour over 24 00, 01, ..., 23
%I | Hour over 12 01, 02, ..., 12
%p | AM or PM
%M | Minute on 2 digits 00, 01, ..., 59
%S | Second on 2 digits 00, 01, ..., 59
%f | Microsecond on 6 digits 000000, 000001, ..., 999999
%z | UTC offset +HHMM or -HHMM
%Z | Time zone 
%c | Representation date and temps
%x | Representation date
%X | Representation time
%% | Character %

In [ ]:
# datetime
now = datetime.datetime.now()
now

In [ ]:
# jour de la semaine
now.strftime("%A %d/%m/%Y")

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
<li>Afficher la date du jour dans le format: `AAAA-MM-JJ/HH:MM:SS`</li>
</ul>
</div>

Le module `locale` permet de gérer l'affichage selon la langue, par défaut en anglais.

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'fr') # français
#locale.setlocale(locale.LC_ALL, 'fr_FR') # français
#locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8') # français

print(now.strftime("%A %d/%m/%Y"))

#locale.setlocale(locale.LC_ALL, 'en') # anglais
#locale.setlocale(locale.LC_ALL, 'en_US') # anglais

Inversement la fonction `strptime()` prend une chaine de caractères et un format de date en arguments et retourne un objet temporel.

In [ ]:
# lecture à partir du format: AA-MM-DD-HH-MM
# écriture dans le format : HH:MM DD/MM/YYYY
var = datetime.datetime.strptime("20-03-26-15-00", "%y-%m-%d-%H-%M")
var.strftime("%H:%M %d/%m/%Y")

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
<li>Lire une date au format ISO : `AAAA-MM-JJTHH:MM:SS`, par exemple 2020-03-26T15:00:00</li>
</ul>
</div>

<div class="alert alert-success">
<b>Exercice 3</b>
<ul>
    <li>Charger le fichier 'cities500.zip', combien y a-t-il de fuseaux horaires différents ?</li>
    <li>Convertir la colonne 'timezone' en objets de type fuseau horaire.</li>
    <li>Prendre l'horodatage du jour et la convertir selon chaque fuseau horaire et produire une chaine au format : '%Y-%m-%d %H:%M:%S'.</li>
    <li> Il y a 24 heure dans une journée, combien y a-t-il de fuseaux horaires différents ?</li>
</ul>
</div>

In [ ]:
# cities500.txt
# http://download.geonames.org/export/dump/cities500.zip
df = pd.read_csv('cities500.zip',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'],
                   dtype={'admin1 code': str, 'admin2 code': str, 'admin3 code': str, 'admin4 code': str},
                   keep_default_na=False,
                   na_values=['', -9999])
df.head()

## 10.3 Séries temporelles

Le librairie `pandas` gère les séries temporelles.

Nous allons introduire quelques notions :
- lecture de données temporelles
- accès aux données temporelles
- calcul d'agrégats temporels
- graphiques et fenêtres glissantes

### Chargement et préparation des données

In [ ]:
# source:
# https://www.banque-france.fr/statistiques/taux-et-cours/les-taux-de-change-salle-des-marches/parites-quotidiennes

# chargement du fichier
tx_change = pd.read_csv("Webstat_Export.csv",
                        sep=";",
                        na_values='-',
                        decimal=',',
                        skiprows=[1, 2],  # le skiprows permet à l'option "decimal" de fonctionner
                        usecols=range(43),
                        converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})
tx_change

#### Préparation des données :

- extraction des codes ISO des devises
- sélection de quelques devises
- drop NaN

In [ ]:
# extraction des codes monnaies
cols = pd.Series(tx_change.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=False)
tx_change.columns = ['Date'] + list(cols[1:])

# sélection de quelques colonnes monnaies
currencies = ['USD', 'CHF', 'GBP', 'JPY', 'RUB', 'CNY']
tx_change = tx_change[['Date'] + currencies]

# drop na
tx_change = tx_change.dropna()

tx_change

In [ ]:
# info
tx_change.info()

### L'opérateur `dt`

L'opérateur `dt` se comporte de manière analogue à l'opérateur `str` pour les `Series`. Il permet d'accéder aux propriététs temporelles des éléments.

In [ ]:
# date
tx_change['Date']

In [ ]:
# les valeurs sont des numpy.datetime64
tx_change['Date'].values[0]

In [ ]:
# accès à l'année
tx_change['Date'].dt.year

In [ ]:
# accès au mois
tx_change['Date'].dt.month

In [ ]:
# access aux jours de la semaine
tx_change['Date'].dt.weekday

In [ ]:
# valeurs uniques
tx_change['Date'].dt.weekday.unique()

### Sélection de données temporelles

Lorsque l'index est un objet de type `datetime`, il est possible d'opérer des sélections en utilisant des dates représentées par des chaines de caractères.

In [ ]:
# setting the index as a datetime object
tx_change = tx_change.set_index('Date')
tx_change = tx_change.sort_index()
tx_change

In [ ]:
# année entière
tx_change.loc['2021']

In [ ]:
# un mois d'une année
tx_change.loc['2021/10']  # ou tx_change.loc['10/2021']

In [ ]:
# sélection de slices - la seconde période est incluse
tx_change['12/2020':'02/2021']

### Complétion de données temporelles

In [ ]:
# complétion des données sur une année
idx = pd.date_range(tx_change.index[0], tx_change.index[-1], freq='D')
tx_change_full = tx_change.reindex(idx)
tx_change_full

In [ ]:
# forward filling
tx_change_full.fillna(method='ffill')

In [ ]:
# backward filling
tx_change_full.fillna(method='bfill')

In [ ]:
# interpolate
tx_change_full.interpolate()

In [ ]:
# numpy.linspace
np.linspace(0, 100, 21)

In [ ]:
# numpy.linspace
arr = np.linspace(tx_change_full.iloc[1], tx_change_full.iloc[4], 4)
pd.DataFrame(arr)

### Agrégations temporelles

Il est possible d'agréger les données selon une certain période de temps en utilisant la méthode `resample()` avec un symbol signifiant la période considérée en ensuite appliquer une méthode d'agrégat.

La méthode `resample()` agit de manière similaire à la méthode `groupby()`.

In [ ]:
tx_change.info()

In [ ]:
# agrégation annuelle
tx_change.resample('A').size()

In [ ]:
# agrégation annuelle
tx_change.resample('A').max()

In [ ]:
# agrégation mensuelle
tx_change.resample('M').mean()

#### fréquences symboliques utilisables (extrait)

Alias | Offset type | Description
- | - | -
D | Day | Calendar daily
B | BusinessDay | Business daily
H | Hour | Hourly
T or min | Minute | Minutely
S | Second | Secondly
L or ms | Milli | Millisecond (1/1000th of 1 second)
U | Micro | Microsecond (1/1000000th of 1 second)
M | MonthEnd | Last calendar day of month
BM | BusinessMonthEnd | Last business day (weekday) of month
MS | MonthBegin | First calendar day of month
BMS | BusinessMonthBegin | First weekday of month
W-MON, W-TUE, ... | Week | Weekly on given day of week: MON, TUE, WED, THU, FRI, SAT, or SUN.
Q-JAN, Q-FEB, ... | QuarterEnd | Quarterly dates anchored on last calendar day of each month,for year ending in indicated month: JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC.
A-JAN, A-FEB, ... | YearEnd | Annual dates anchored on last calendar day of given month: JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC.

Source: Python for Data Analysis, Wes McKinney, O'Reilly

### Graphiques temporels

Il est possible de produire directement des graphiques.

In [ ]:
# taux de change
tx_change.plot();

Ci-dessous on divise les taux par leur moyenne respective afin d'ajuster les échelles et qu'elles soient comparables.

In [ ]:
# taux de change divisés par leurs moyennes respectives
(tx_change / tx_change.mean()).plot();

La méthode `rolling()` permet de produire un graphique avec une moyenne mobile par exemple.

In [ ]:
# taux de change divisés par leurs moyennes respectives avec une moyenne mobile de 30 jours
(tx_change / tx_change.mean()).rolling(30).mean().plot();

<div class="alert alert-success">
<b>Exercice 4</b>
<ul>
    <li>Afficher un graphique des taux de change divisés par leurs dernières valeurs</li>
    <li>Afficher un graphique des taux de change divisés par leurs dernières valeurs avec une moyenne mobile de 30 jours</li>
    <li>Afficher un graphique des taux de change divisés par leurs moyennes respectives avec un maximum mobile de 100 jours</li>
</ul>
</div>